In [1]:
import os
import sys
import math
import time
import IPython
import logging
import warnings
import datetime
import pandas as pd
import numpy  as np
import webbrowser
# import scipy  as sp
import pprint as pp
from random   import randrange
from notebook import notebookapp
from IPython.display import display
from IPython.core.display import HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
sys.path.insert(1, os.path.join(sys.path[0], '{}/lib/'.format(os.environ['BaseDir'])))
from corda5Interface import Corda5
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

apiServer='https://localhost:8888/api/v1/flow'
apiServerAuth=('admin', 'admin')
package = 'com.r3.developers.configurableInstrument'

nodes = {
        'Authority' : {'x500': 'CN=Authority, OU=Test Dept, O=R3, L=London, C=GB' , 'hash': '8175C6EA55DE' },
        'Bob'       : {'x500': 'CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'       , 'hash': 'B31CAD5945F1' },
        'Charlie'   : {'x500': 'CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'   , 'hash': 'AB7643993711' },
        'Alice'     : {'x500': 'CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'     , 'hash': '415E407297AA' },
#         'NotaryRep1': {'x500': 'CN=NotaryRep1, OU=Test Dept, O=R3, L=London, C=GB', 'hash': '2F9FE806D09A' },
#         'Dave'      : {'x500': 'CN=Dave, OU=Test Dept, O=R3, L=London, C=GB'      , 'hash': 'FB9F8A560956' },
        }
replace_maters_list = { 'API_SERVER': apiServer, 'PACKAGE': package}

# h = JSON_ON_Jupyter(level=logging.DEBUG, nodes=nodes, apiServer=apiServer, apiServerAuth=apiServerAuth, version=2)
h = Corda5(level=logging.INFO, nodes=nodes, apiServer=apiServer, apiServerAuth=apiServerAuth, version=2)

log = h.log
# def set_in_master_list(k, v): replace_maters_list[k] = v

# https://localhost:8888/api/v1/flow/{{Dave Hash}}/30020

ModuleNotFoundError: No module named 'corda5Interface'

In [2]:
print(os.environ['BaseDir'])

/Users/Manoj.Gupta/git/c5_hackathon_beta3/python3/jupyter


In [2]:
print("1) Issue -- Type 1, Transferable = Y, Quantifiable = Y")
print("2) Issue -- Type 2, Transferable = Y, Quantifiable = N")
print("3) Issue -- Type 3, Transferable = N, Quantifiable = Y")
print("4) Issue -- Type 4, Transferable = N, Quantifiable = N")

def message(text):
    if type(text) == list:
        for t in text:
            print(t, end="")
    else:
        print(text)

1) Issue -- Type 1, Transferable = Y, Quantifiable = Y
2) Issue -- Type 2, Transferable = Y, Quantifiable = N
3) Issue -- Type 3, Transferable = N, Quantifiable = Y
4) Issue -- Type 4, Transferable = N, Quantifiable = N


In [ ]:
# Issue -- Type 1, Transferable = Y, Quantifiable = Y
# Example 1 -- Government Bond with defined coupon payments -- Type 1 / Test 1 -- Transferable, Quantifiable Instrument
(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'Government Bond 2024', 
        quantity=10,
        transferable=True,
        expiry=None,
        verifiable=True,
        attributes={'payments': "['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']"})

message([req_id, response, return_val_uuid])

 15-05 19:00:36 | I | Issuing instrument='Government Bond 2024' to 'Alice'
 15-05 19:00:36 | I | Running action 'Issue' on 'Authority/8175C6EA55DE/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'


In [3]:
help(h.transfer)

13652.1683781132.6881862 <Response [200]> be8f210d-b376-418c-9c0f-b051ce125a33


In [5]:
# Transfer from issuer -- expected=failure
(req_id, response, return_val) = h.transfer(return_val_uuid, 'Authority', 'Charlie', quantity=4)
message([req_id, response, return_val])

 15-05 11:00:41 | I | Tranfering instrument id '15afb175-a785-4747-bb22-6f5431d8a61e' to 'Charlie'
 15-05 11:00:41 | I | Running action 'Transfer' on 'Authority/8175C6EA55DE/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 15-05 11:00:45 | I | {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id 15afb175-a785-4747-bb22-6f5431d8a61e'}


28810.1684128641.0943942 <Response [200]> {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id 15afb175-a785-4747-bb22-6f5431d8a61e'}


In [8]:
# Partial transfer from owner
(req_id, response, return_val) = h.transfer(return_val_uuid, 'Alice', 'Bob', quantity=6)
message([req_id, response, return_val])

 15-05 17:25:38 | I | Tranfering instrument id '15afb175-a785-4747-bb22-6f5431d8a61e' to 'Bob'
 15-05 17:25:38 | I | Running action 'Transfer' on 'Alice/415E407297AA/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 15-05 17:26:47 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@6a00742c


TypeError: message() takes 1 positional argument but 3 were given

In [11]:
# Full Transfer from owner/holder -- expected=success
(req_id, response, return_val) = h.transfer(return_val_uuid, 'Alice', 'Charlie', quantity=4)
message([req_id, response, return_val])

 15-05 17:33:41 | I | Tranfering instrument id '15afb175-a785-4747-bb22-6f5431d8a61e' to 'Charlie'
 15-05 17:33:41 | I | Running action 'Transfer' on 'Alice/415E407297AA/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 15-05 17:34:58 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@6bfe1c77


28810.1684152221.3185458<Response [200]>net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@6bfe1c77

In [9]:
# Transfer attempt from non-holder -- expected=failure
(req_id, response, return_val) = h.transfer(return_val_uuid, 'Alice', 'Charlie', quantity=4)

 11-05 10:54:01 | I | Tranfering instrument id 'be8f210d-b376-418c-9c0f-b051ce125a33' to 'Bob'
 11-05 10:54:01 | I | Running action 'Transfer' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 11-05 10:54:06 | I | {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id be8f210d-b376-418c-9c0f-b051ce125a33'}


In [21]:
# Redeem
help(h.redeem)

Help on method redeem in module json_on_jupyter:

redeem(inst_id, from_node, quantity=None) method of json_on_jupyter.JSON_ON_Jupyter instance



In [2]:
h.query('Alice')

""


In [24]:
# Redeem - Partial by holder -- exepected=success
h.redeem('8f90d61f-5380-417a-a647-1194f4033b01', 'Bob', quantity=1)

 11-05 18:02:48 | I | Redeeming instrument id '8f90d61f-5380-417a-a647-1194f4033b01' quantity '1'
 11-05 18:02:48 | I | Running action 'Redeem' on 'Bob/83214F5048CD/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'
 11-05 18:03:41 | I | UtxoSignedTransactionImpl(id=SHA-256D:39AEE0AB718BFA57EB45ED5EAFC6B7310BE8E52E4B1B8CA22D18FE7D31E5E7E8, signatures=[DigitalSignatureAndMetadata[signature=[3044022068E89A43D1E511F269CF9DAF9F4A20304CADB533842605D5D6A87BBA8F50A2F40220374D78E5B8BACFF4F423A6498F9B9C328630E35188D9A8E85A4AA7D2746171B5], metadata=DigitalSignatureMetadata[timestamp=2023-05-11T12:32:54.621327Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=null], DigitalSignatureAndMetadata[signature=[3044022066F92D6E13224E1D9F923C1C6570CA26E8A706843315777BB9DA96E356FC5B8802203D0AF3C7EF13D47F55F90599A1B28BF855124FB286921BC4EF0C6B1ECB1BBD7E], metadata=DigitalSignatureMetadata[timestamp=2023-05

('13652.1683808368.2703528',
 <Response [200]>,
 'UtxoSignedTransactionImpl(id=SHA-256D:39AEE0AB718BFA57EB45ED5EAFC6B7310BE8E52E4B1B8CA22D18FE7D31E5E7E8, signatures=[DigitalSignatureAndMetadata[signature=[3044022068E89A43D1E511F269CF9DAF9F4A20304CADB533842605D5D6A87BBA8F50A2F40220374D78E5B8BACFF4F423A6498F9B9C328630E35188D9A8E85A4AA7D2746171B5], metadata=DigitalSignatureMetadata[timestamp=2023-05-11T12:32:54.621327Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=null], DigitalSignatureAndMetadata[signature=[3044022066F92D6E13224E1D9F923C1C6570CA26E8A706843315777BB9DA96E356FC5B8802203D0AF3C7EF13D47F55F90599A1B28BF855124FB286921BC4EF0C6B1ECB1BBD7E], metadata=DigitalSignatureMetadata[timestamp=2023-05-11T12:33:10.806098Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=n

In [36]:
# Redeem - Partial by Issuer -- exepected=success
h.redeem('be8f210d-b376-418c-9c0f-b051ce125a33', 'Authority', quantity=1)

 11-05 18:54:15 | I | Redeeming instrument id 'be8f210d-b376-418c-9c0f-b051ce125a33' quantity '1'
 11-05 18:54:15 | I | Running action 'Redeem' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 11-05 18:55:16 | I | UtxoSignedTransactionImpl(id=SHA-256D:FEE9545D50015115BEA3B7C13001FCC9CEB7BA45A73F9F2CB9C2823D709BF403, signatures=[DigitalSignatureAndMetadata[signature=[30460221009182B3F47C0D9F58547590BE425184CEB7D5978A6DC5C981B036EBE32D4D99B0022100D6B40D5C3AD082F3D3562DA9E77898078F82DDD4837E52818E7E5B146BB86616], metadata=DigitalSignatureMetadata[timestamp=2023-05-11T13:24:25.326126Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=null], DigitalSignatureAndMetadata[signature=[30450220689F998A3E8E3E2EEC5198FB3AEB94E6C0E49E7F2E2E63B1811B105C80D7F221022100B3A0CE17172717BB290CE8543ED24D21012FC2F741AEC3A809D3A016B9835FBE], metadata=DigitalSignatureMetadata

('13652.1683811455.264299',
 <Response [200]>,
 'UtxoSignedTransactionImpl(id=SHA-256D:FEE9545D50015115BEA3B7C13001FCC9CEB7BA45A73F9F2CB9C2823D709BF403, signatures=[DigitalSignatureAndMetadata[signature=[30460221009182B3F47C0D9F58547590BE425184CEB7D5978A6DC5C981B036EBE32D4D99B0022100D6B40D5C3AD082F3D3562DA9E77898078F82DDD4837E52818E7E5B146BB86616], metadata=DigitalSignatureMetadata[timestamp=2023-05-11T13:24:25.326126Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=null], DigitalSignatureAndMetadata[signature=[30450220689F998A3E8E3E2EEC5198FB3AEB94E6C0E49E7F2E2E63B1811B105C80D7F221022100B3A0CE17172717BB290CE8543ED24D21012FC2F741AEC3A809D3A016B9835FBE], metadata=DigitalSignatureMetadata[timestamp=2023-05-11T13:24:46.185261Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], pr

In [37]:
h.query('Authority')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,8f90d61f-5380-417a-a647-1194f4033b01,Government Bond 2024,"CN=Bob, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",1,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"
1,be8f210d-b376-418c-9c0f-b051ce125a33,Government Bond 2024,"CN=Bob, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",3,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"


In [40]:
# Redeem - Full by holder -- exepected=success
h.redeem('be8f210d-b376-418c-9c0f-b051ce125a33', 'Bob', quantity=3)

 11-05 19:22:05 | I | Redeeming instrument id 'be8f210d-b376-418c-9c0f-b051ce125a33' quantity '3'
 11-05 19:22:05 | I | Running action 'Redeem' on 'Bob/83214F5048CD/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'
 11-05 19:23:02 | I | UtxoSignedTransactionImpl(id=SHA-256D:D4DC0A91F99E0973C770A4E9B1B422AEE6BA4B1C9F1AB2BEBE29C2B909D99FF9, signatures=[DigitalSignatureAndMetadata[signature=[3046022100A76E7F8134BF4707BE2460E00C9B05E023EBDADF01F5D8304060F22158E9FE57022100FE0527B885912D31D6888DBF90C5BC8A0A228D034E1B19080483C2DB6A799B2D], metadata=DigitalSignatureMetadata[timestamp=2023-05-11T13:52:12.853107Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=null], DigitalSignatureAndMetadata[signature=[30450220632BF51560B3AA562EFF980B6423D07F517C67B8D065D608D5150A20552FAA89022100E3BAF15335A8B8C2159695048865B588A62D111D051C696EA8BD9078DCD51336], metadata=DigitalSignatureMetadata[timestamp=2

('13652.1683813125.7133381',
 <Response [200]>,
 'UtxoSignedTransactionImpl(id=SHA-256D:D4DC0A91F99E0973C770A4E9B1B422AEE6BA4B1C9F1AB2BEBE29C2B909D99FF9, signatures=[DigitalSignatureAndMetadata[signature=[3046022100A76E7F8134BF4707BE2460E00C9B05E023EBDADF01F5D8304060F22158E9FE57022100FE0527B885912D31D6888DBF90C5BC8A0A228D034E1B19080483C2DB6A799B2D], metadata=DigitalSignatureMetadata[timestamp=2023-05-11T13:52:12.853107Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=null], DigitalSignatureAndMetadata[signature=[30450220632BF51560B3AA562EFF980B6423D07F517C67B8D065D608D5150A20552FAA89022100E3BAF15335A8B8C2159695048865B588A62D111D051C696EA8BD9078DCD51336], metadata=DigitalSignatureMetadata[timestamp=2023-05-11T13:52:31.371126Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], p

In [38]:
h.query('Bob')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,8f90d61f-5380-417a-a647-1194f4033b01,Government Bond 2024,"CN=Bob, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",5,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"
1,be8f210d-b376-418c-9c0f-b051ce125a33,Government Bond 2024,"CN=Bob, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",3,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"


In [30]:
h.query_all_nodes() # calling flow = ListInstrument

# for n in nodes:
#     df = h.query(n, query, False)
#     h.log.info("Result for '{}' from '{}/{}/{}'".format(query, n, nodes[n]['x500'], nodes[n]['hash']))
#     display(df)

 10-05 11:18:38 | D | Running query=ListInstrument on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 10-05 11:18:38 | D | Request type=post to url='https://localhost:8888/api/v1/flow/069C8B8CD60D'
 10-05 11:18:38 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.query.__QUERY__', 'requestBody': {}}
 10-05 11:18:38 | D | {'clientRequestId': '76808.1683697718.418418', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 10-05 11:18:38 | D | After filling values = {'clientRequestId': '76808.1683697718.418418', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 10-05 11:18:38 | D | 202
 10-05 11:18:38 | D | {'holdingIdentityShortHash': '069C8B8CD60D', 'clientRequestId': '76808.1683697718.418418', 'flowId': None, 'flowStatus': 'START_REQUESTED', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-10T05:48:38.4

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,9339350b-d16e-4a94-91ac-e363902172a8,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",0,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"
1,104ae922-d085-4b29-8e24-6014057f1a50,eRupee,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",100001,True,None,True,{}


 10-05 11:18:42 | D | Running query=ListInstrument on 'Bob/83214F5048CD/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'
 10-05 11:18:42 | D | Request type=post to url='https://localhost:8888/api/v1/flow/83214F5048CD'
 10-05 11:18:42 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.query.__QUERY__', 'requestBody': {}}
 10-05 11:18:42 | D | {'clientRequestId': '76808.1683697722.625798', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 10-05 11:18:42 | D | After filling values = {'clientRequestId': '76808.1683697722.625798', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 10-05 11:18:42 | D | 202
 10-05 11:18:42 | D | {'holdingIdentityShortHash': '83214F5048CD', 'clientRequestId': '76808.1683697722.625798', 'flowId': None, 'flowStatus': 'START_REQUESTED', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-10T05:48:42.647Z'}
 10-05

""


 10-05 11:18:46 | D | Running query=ListInstrument on 'Charlie/D728EEE9116F/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'
 10-05 11:18:46 | D | Request type=post to url='https://localhost:8888/api/v1/flow/D728EEE9116F'
 10-05 11:18:46 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.query.__QUERY__', 'requestBody': {}}
 10-05 11:18:46 | D | {'clientRequestId': '76808.1683697726.758862', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 10-05 11:18:46 | D | After filling values = {'clientRequestId': '76808.1683697726.758862', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 10-05 11:18:46 | D | 202
 10-05 11:18:46 | D | {'holdingIdentityShortHash': 'D728EEE9116F', 'clientRequestId': '76808.1683697726.758862', 'flowId': None, 'flowStatus': 'START_REQUESTED', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-10T05:48:46.775Z'

""


 10-05 11:18:50 | D | Running query=ListInstrument on 'Alice/98A345A3E88B/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 10-05 11:18:50 | D | Request type=post to url='https://localhost:8888/api/v1/flow/98A345A3E88B'
 10-05 11:18:50 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.query.__QUERY__', 'requestBody': {}}
 10-05 11:18:50 | D | {'clientRequestId': '76808.1683697730.9354', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 10-05 11:18:50 | D | After filling values = {'clientRequestId': '76808.1683697730.9354', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 10-05 11:18:50 | D | 202
 10-05 11:18:50 | D | {'holdingIdentityShortHash': '98A345A3E88B', 'clientRequestId': '76808.1683697730.9354', 'flowId': None, 'flowStatus': 'START_REQUESTED', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-10T05:48:50.958Z'}
 10-05 1

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,9339350b-d16e-4a94-91ac-e363902172a8,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",5,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"
1,104ae922-d085-4b29-8e24-6014057f1a50,eRupee,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",100001,True,None,True,{}


 09-05 13:20:19 | I | Issuing instrument='Government Bond 2024' to 'Alice'
 09-05 13:20:19 | I | Running action 'Issue' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 13:21:24 | I | 9339350b-d16e-4a94-91ac-e363902172a8


('67712.1683618619.295648',
 <Response [200]>,
 '9339350b-d16e-4a94-91ac-e363902172a8')

In [41]:
# Example 2 -- Digital currency (eRupee) -- Type 1 / Test 1 -- Transferable, Quantifiable Instrument

(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'eRupee',
        quantity=100001,
        transferable=True,
        expiry=None,
        verifiable=True
       )
print(req_id, response, return_val_uuid)

 11-05 19:41:42 | I | Issuing instrument='eRupee' to 'Alice'
 11-05 19:41:42 | I | Running action 'Issue' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 11-05 19:42:38 | I | 3d2ecb60-cdb8-4004-9c79-3cde976969fc


13652.1683814302.192371 <Response [200]> 3d2ecb60-cdb8-4004-9c79-3cde976969fc


In [44]:
# Example 3 -- Unique artifacts on Ledger -- Type 2 / Test 2 -- Transferable, non Quantifiable Instrument
(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'Unique artifacts on Ledger. The Mona Lisa', 
        quantity=None,
        transferable=True,
        expiry=None,
        verifiable=True,
        attributes={'holding autority': 'The Louvre Museum, Paris'}
       )
print(req_id, response, return_val_uuid)

 11-05 20:01:26 | I | Issuing instrument='Unique artifacts on Ledger. The Mona Lisa' to 'Alice'
 11-05 20:01:26 | I | Running action 'Issue' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 11-05 20:02:27 | I | aae1090f-beea-4341-a342-1056d02f98ab


In [ ]:
# another example  -- Limited Edition Soverign -- Type 1 / Test 1 -- Transferable, Quantifiable Instrument

In [45]:
# Example 4 -- Fiat currency with unique number in digital form -- Type 2 / Test 2 -- Transferable, non Quantifiable Instrument
# - Issue Instrument transferable=True, quantity=null(non aggregable, even if quantity is 1, still since id is different, it won't aggregage)
#     example "Digital Currency INR 500 Note", attributes{'id': "OMV 336048", 'year': '', '', ''}
(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'Digital Currency INR 500 Note', 
        quantity=None, # quantity is one or None its the same
        transferable=True,
        expiry=None,
        verifiable=True, 
        attributes={'number': 'OMV 336048', 'year': '1981'}
       )
print(req_id, response, return_val_uuid)

 11-05 20:02:27 | I | Issuing instrument='Digital Currency INR 500 Note' to 'Alice'
 11-05 20:02:27 | I | Running action 'Issue' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 11-05 20:03:19 | I | 5819d61a-5d73-44ca-b677-7125ebc8bf97


In [46]:
print(req_id, response, return_val_uuid)

13652.1683815547.127303 <Response [200]> 5819d61a-5d73-44ca-b677-7125ebc8bf97


In [28]:
# Example 5 -- Travel Points, Redeemable Tokens (Type 3 / Test 3 -- Non transferable, Quantifiable Instrument)
h.issue('Authority', 'Alice', 'Star Alliance Travel Points', quantity=10, transferable=False, expiry=None, verifiable=True)

 09-05 22:57:41 | I | Issuing instrument='Star Alliance Travel Points' to 'Alice'
 09-05 22:57:41 | I | Running action 'Issue' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 22:57:41 | D | Request type=post to url='https://localhost:8888/api/v1/flow/069C8B8CD60D'
 09-05 22:57:41 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.workflows.Issue', 'requestBody': {'name': 'Star Alliance Travel Points', 'to': 'CN=Alice, OU=Test Dept, O=R3, L=London, C=GB', 'transferable': 'false', 'verifiable': 'true', 'quantity': 10}}
 09-05 22:57:41 | D | {'clientRequestId': '76808.1683653261.445578', 'flowClassName': 'com.r3.developers.configurableInstrument.workflows.Issue', 'requestBody': {'name': 'Star Alliance Travel Points', 'to': 'CN=Alice, OU=Test Dept, O=R3, L=London, C=GB', 'transferable': 'false', 'verifiable': 'true', 'quantity': 10}}
 09-05 22:57:41 | D | After filling values = {'clientRequestId': '76808.1683

 09-05 23:45:54 | D | Request type=get to url='https://localhost:8888/api/v1/flow/069C8B8CD60D/76808.1683653261.445578'
 09-05 23:45:54 | D | 200
 09-05 23:45:54 | D | {'holdingIdentityShortHash': '069C8B8CD60D', 'clientRequestId': '76808.1683653261.445578', 'flowId': '251611e4-83dd-426d-87fa-493c0358abda', 'flowStatus': 'RUNNING', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-09T17:27:41.891Z'}
 10-05 00:03:28 | D | Request type=get to url='https://localhost:8888/api/v1/flow/069C8B8CD60D/76808.1683653261.445578'
 10-05 00:03:28 | D | 200
 10-05 00:03:28 | D | {'holdingIdentityShortHash': '069C8B8CD60D', 'clientRequestId': '76808.1683653261.445578', 'flowId': '251611e4-83dd-426d-87fa-493c0358abda', 'flowStatus': 'RUNNING', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-09T17:27:41.891Z'}
 10-05 00:03:32 | D | Request type=get to url='https://localhost:8888/api/v1/flow/069C8B8CD60D/76808.1683653261.445578'
 10-05 00:03:32 | D | 200
 10-05 00:03:33 | D | {'ho

 10-05 03:24:17 | D | Request type=get to url='https://localhost:8888/api/v1/flow/069C8B8CD60D/76808.1683653261.445578'
 10-05 03:24:17 | D | 200
 10-05 03:24:17 | D | {'holdingIdentityShortHash': '069C8B8CD60D', 'clientRequestId': '76808.1683653261.445578', 'flowId': '251611e4-83dd-426d-87fa-493c0358abda', 'flowStatus': 'RUNNING', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-09T17:27:41.891Z'}
 10-05 03:42:14 | D | Request type=get to url='https://localhost:8888/api/v1/flow/069C8B8CD60D/76808.1683653261.445578'
 10-05 03:42:14 | D | 200
 10-05 03:42:14 | D | {'holdingIdentityShortHash': '069C8B8CD60D', 'clientRequestId': '76808.1683653261.445578', 'flowId': '251611e4-83dd-426d-87fa-493c0358abda', 'flowStatus': 'RUNNING', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-09T17:27:41.891Z'}
 10-05 03:42:18 | D | Request type=get to url='https://localhost:8888/api/v1/flow/069C8B8CD60D/76808.1683653261.445578'
 10-05 03:42:18 | D | 200
 10-05 03:42:18 | D | {'ho

('76808.1683653261.445578',
 <Response [200]>,
 "Flow failed, message: Type='PLATFORM_ERROR' Message='Failed to send: 1 of 1 sessions are invalid ['1107bd8e-c58e-4ab7-9e88-dbae20503292'=ERROR]. Please check CorDapp protocol expectations match on both sides regarding 'send' and 'receive' statements and that sessions are not closed unexpectedly on either side. Stack traces around this error originating in CorDapp code may provide a hint.'")

In [9]:
# Example 6 -- Amazon Vouchers (# Type 3 / Test 3 -- Non transferable, Quantifiable Instrument)
h.issue('Authority', 'Alice', 'Amazon Voucher', quantity=2000, transferable=False, expiry=None, verifiable=True, attributes={'currency': 'INR'})

 09-05 11:18:48 | I | Issuing instrument='Amazon Voucher' to 'Alice'
 09-05 11:18:48 | I | Running action 'Issue' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 11:19:49 | I | 239ca025-092e-4b0b-a1e5-1fca6003b657


('54796.1683611328.611851',
 <Response [200]>,
 '239ca025-092e-4b0b-a1e5-1fca6003b657')

In [14]:
# Example 7 - Graduation certificate (Type 4 / Test 4 -- Non transferable, Non quantifiable Instrument)
h.issue('Authority', 'Charlie', 'BE Computer Engineering', quantity=None, transferable=False, expiry=None, verifiable=True,
        attributes={
            'university': 'University of Mumbai', 
            'year': '2023',
            'score': '450/500',
            'major subject': "['Robotics', 'AI', 'DSP']"})

 09-05 11:21:37 | I | Issuing instrument='BE Computer Engineering' to 'Charlie'
 09-05 11:21:37 | I | Running action 'Issue' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 11:22:37 | I | 6c764908-fdb0-40f3-af53-18f0159c6897


('54796.1683611497.236011',
 <Response [200]>,
 '6c764908-fdb0-40f3-af53-18f0159c6897')

In [16]:
# Example 8 - Experience/Participation certificate (Type 4 / Test 4 -- Non transferable, Non quantifiable Instrument)
h.issue('Authority', 'Bob', 'R3 Hackathon 2023', quantity=None, transferable=False, expiry=None, verifiable=True, attributes={'Project': 'Corda 5', 'Year': '2023'})

 09-05 11:23:16 | I | Issuing instrument='R3 Hackathon 2023' to 'Bob'
 09-05 11:23:16 | I | Running action 'Issue' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 11:24:17 | I | 2af47ace-c52c-4237-b336-b2694e75494a


('54796.1683611596.371066',
 <Response [200]>,
 '2af47ace-c52c-4237-b336-b2694e75494a')

In [17]:
h.query_all_nodes() # calling flow = ListInstrument

 09-05 16:08:34 | D | Running query=ListInstrument on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 16:08:34 | D | Request type=post to url='https://localhost:8888/api/v1/flow/069C8B8CD60D'
 09-05 16:08:34 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.query.__QUERY__', 'requestBody': {}}
 09-05 16:08:34 | D | {'clientRequestId': '76808.1683628714.729872', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 09-05 16:08:34 | D | After filling values = {'clientRequestId': '76808.1683628714.729872', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 09-05 16:08:34 | D | 202
 09-05 16:08:34 | D | {'holdingIdentityShortHash': '069C8B8CD60D', 'clientRequestId': '76808.1683628714.729872', 'flowId': None, 'flowStatus': 'START_REQUESTED', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-09T10:38:34.7

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,9339350b-d16e-4a94-91ac-e363902172a8,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",2,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"


 09-05 16:08:38 | D | Running query=ListInstrument on 'Bob/83214F5048CD/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 16:08:38 | D | Request type=post to url='https://localhost:8888/api/v1/flow/83214F5048CD'
 09-05 16:08:38 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.query.__QUERY__', 'requestBody': {}}
 09-05 16:08:38 | D | {'clientRequestId': '76808.1683628718.856797', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 09-05 16:08:38 | D | After filling values = {'clientRequestId': '76808.1683628718.856797', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 09-05 16:08:38 | D | 202
 09-05 16:08:38 | D | {'holdingIdentityShortHash': '83214F5048CD', 'clientRequestId': '76808.1683628718.856797', 'flowId': None, 'flowStatus': 'START_REQUESTED', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-09T10:38:38.877Z'}
 09-05

""


 09-05 16:08:43 | D | Running query=ListInstrument on 'Charlie/D728EEE9116F/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 16:08:43 | D | Request type=post to url='https://localhost:8888/api/v1/flow/D728EEE9116F'
 09-05 16:08:43 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.query.__QUERY__', 'requestBody': {}}
 09-05 16:08:43 | D | {'clientRequestId': '76808.1683628723.005763', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 09-05 16:08:43 | D | After filling values = {'clientRequestId': '76808.1683628723.005763', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 09-05 16:08:43 | D | 202
 09-05 16:08:43 | D | {'holdingIdentityShortHash': 'D728EEE9116F', 'clientRequestId': '76808.1683628723.005763', 'flowId': None, 'flowStatus': 'START_REQUESTED', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-09T10:38:43.027Z'

""


 09-05 16:08:47 | D | Running query=ListInstrument on 'Alice/98A345A3E88B/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 16:08:47 | D | Request type=post to url='https://localhost:8888/api/v1/flow/98A345A3E88B'
 09-05 16:08:47 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.query.__QUERY__', 'requestBody': {}}
 09-05 16:08:47 | D | {'clientRequestId': '76808.1683628727.147019', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 09-05 16:08:47 | D | After filling values = {'clientRequestId': '76808.1683628727.147019', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 09-05 16:08:47 | D | 202
 09-05 16:08:47 | D | {'holdingIdentityShortHash': '98A345A3E88B', 'clientRequestId': '76808.1683628727.147019', 'flowId': None, 'flowStatus': 'START_REQUESTED', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-09T10:38:47.171Z'}
 0

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,9339350b-d16e-4a94-91ac-e363902172a8,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",5,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"


In [ ]:
df = h.query('Charlie', 'ListInstrument')

In [22]:
df = h.query('Authority')
# inst_id='7e7bd79d-d5d6-4c9d-954b-9a47e2dc517d'
# tdf = df[df['id'] == inst_id]
# tdf.transpose()
# tdf.to_dict('records')[0]
# del(tdf['id'])

 09-05 22:53:46 | D | Running query=ListInstrument on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 22:53:46 | D | Request type=post to url='https://localhost:8888/api/v1/flow/069C8B8CD60D'
 09-05 22:53:46 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.query.__QUERY__', 'requestBody': {}}
 09-05 22:53:46 | D | {'clientRequestId': '76808.1683653026.647611', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 09-05 22:53:46 | D | After filling values = {'clientRequestId': '76808.1683653026.647611', 'flowClassName': 'com.r3.developers.configurableInstrument.query.ListInstrument', 'requestBody': {}}
 09-05 22:53:46 | D | 202
 09-05 22:53:46 | D | {'holdingIdentityShortHash': '069C8B8CD60D', 'clientRequestId': '76808.1683653026.647611', 'flowId': None, 'flowStatus': 'START_REQUESTED', 'flowResult': None, 'flowError': None, 'timestamp': '2023-05-09T17:23:46.6

In [23]:
df

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,9339350b-d16e-4a94-91ac-e363902172a8,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",0,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"


In [21]:
h.redeem('9339350b-d16e-4a94-91ac-e363902172a8', from_node='Authority', quantity=2)

 09-05 22:52:19 | I | Redeeming instrument id '9339350b-d16e-4a94-91ac-e363902172a8' quantity '2'
 09-05 22:52:19 | I | Running action 'Redeem' on 'Authority/069C8B8CD60D/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 22:52:19 | D | Request type=post to url='https://localhost:8888/api/v1/flow/069C8B8CD60D'
 09-05 22:52:19 | D | Original Request = {'clientRequestId': '__REQUEST_NUMBER__', 'flowClassName': '__PACKAGE__.workflows.Redeem', 'requestBody': {'id': '__REDEEM_ASSET_ID__', 'quantity': 2}}
 09-05 22:52:19 | D | {'clientRequestId': '76808.1683652939.960599', 'flowClassName': 'com.r3.developers.configurableInstrument.workflows.Redeem', 'requestBody': {'id': '9339350b-d16e-4a94-91ac-e363902172a8', 'quantity': 2}}
 09-05 22:52:19 | D | After filling values = {'clientRequestId': '76808.1683652939.960599', 'flowClassName': 'com.r3.developers.configurableInstrument.workflows.Redeem', 'requestBody': {'id': '9339350b-d16e-4a94-91ac-e363902172a8', 'quantity': 2}}
 09-05 22:52:20 

 09-05 22:53:12 | I | UtxoSignedTransactionImpl(id=SHA-256D:5A97A6A78890E68C79B2EF0DA3B063B9C48A5DE039CC748E27DCB5DB5F6306A7, signatures=[DigitalSignatureAndMetadata[signature=[304502202A2AC7A2A8C5CC8EBD17C31B3C9D9A085125CF305944C695B4C5146C0CD7D4690221009CF759387E0F3F59099F5B71DE3E15473B35CA3837522FFE1928D042792E17F4], metadata=DigitalSignatureMetadata[timestamp=2023-05-09T17:22:27.505471Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=null], DigitalSignatureAndMetadata[signature=[3045022100B53F29E782CB834CA9D9E35EB35E568BA60E3DE909805C7F8885CD68C9337FE70220493E393C4ADC4A3F6E6830555901B4CC04711A180D4DB89247C51C359614B687], metadata=DigitalSignatureMetadata[timestamp=2023-05-09T17:22:56.059090Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565, batchMerkleTreeDigestProviderName

('76808.1683652939.960599',
 <Response [200]>,
 'UtxoSignedTransactionImpl(id=SHA-256D:5A97A6A78890E68C79B2EF0DA3B063B9C48A5DE039CC748E27DCB5DB5F6306A7, signatures=[DigitalSignatureAndMetadata[signature=[304502202A2AC7A2A8C5CC8EBD17C31B3C9D9A085125CF305944C695B4C5146C0CD7D4690221009CF759387E0F3F59099F5B71DE3E15473B35CA3837522FFE1928D042792E17F4], metadata=DigitalSignatureMetadata[timestamp=2023-05-09T17:22:27.505471Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=null], DigitalSignatureAndMetadata[signature=[3045022100B53F29E782CB834CA9D9E35EB35E568BA60E3DE909805C7F8885CD68C9337FE70220493E393C4ADC4A3F6E6830555901B4CC04711A180D4DB89247C51C359614B687], metadata=DigitalSignatureMetadata[timestamp=2023-05-09T17:22:56.059090Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565, batchM

In [2]:
pdf_report = h.print_to_pdf('Alice', 'fb31249e-3696-43f5-99d5-b650862f1e4e', show=False)
webbrowser.open("file:///{}".format(pdf_report))

[WDM] - Downloading: 100%|██████████| 8.04M/8.04M [00:00<00:00, 11.4MB/s]


True

In [10]:
pdf_report = h.print_to_pdf('Charlie', 'ab2a0ff2-92c4-4b73-af50-5fed62790214', show=False)
webbrowser.open("file:///{}".format(pdf_report))

True

In [9]:
h.transfer('6c764908-fdb0-40f3-af53-18f0159c6897', 'Charlie', 'Bob')

 09-05 11:57:24 | I | Tranfering instrument id '6c764908-fdb0-40f3-af53-18f0159c6897' to 'Bob'
 09-05 11:57:24 | I | Running action 'Transfer' on 'Charlie/D728EEE9116F/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'
 09-05 11:57:33 | I | {'type': 'FLOW_FAILED', 'message': 'net.corda.v5.ledger.utxo.ContractVerificationException: Verification of ledger transaction with ID SHA-256D:A2EBCEADF36EB733619F98987E3400B42E75CA93CE2998CF63FB2779D2A303BE failed: net.corda.v5.ledger.utxo.ContractVerificationException: Ledger transaction contract verification failed for the specified transaction: SHA-256D:A2EBCEADF36EB733619F98987E3400B42E75CA93CE2998CF63FB2779D2A303BE.\nThe following contract verification requirements were not met:\ncom.r3.developers.configurableInstrument.contracts.InstrumentContract: Failed requirement: Instrument is not transferable.\n'}


('57457.1683613644.99713',
 <Response [200]>,
 {'type': 'FLOW_FAILED',
  'message': 'net.corda.v5.ledger.utxo.ContractVerificationException: Verification of ledger transaction with ID SHA-256D:A2EBCEADF36EB733619F98987E3400B42E75CA93CE2998CF63FB2779D2A303BE failed: net.corda.v5.ledger.utxo.ContractVerificationException: Ledger transaction contract verification failed for the specified transaction: SHA-256D:A2EBCEADF36EB733619F98987E3400B42E75CA93CE2998CF63FB2779D2A303BE.\nThe following contract verification requirements were not met:\ncom.r3.developers.configurableInstrument.contracts.InstrumentContract: Failed requirement: Instrument is not transferable.\n'})

In [8]:
h.nodes

{'Authority': {'x500': 'CN=Authority, OU=Test Dept, O=R3, L=London, C=GB',
  'hash': '069C8B8CD60D'},
 'Bob': {'x500': 'CN=Bob, OU=Test Dept, O=R3, L=London, C=GB',
  'hash': '83214F5048CD'},
 'Charlie': {'x500': 'CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB',
  'hash': 'D728EEE9116F'},
 'Alice': {'x500': 'CN=Alice, OU=Test Dept, O=R3, L=London, C=GB',
  'hash': '98A345A3E88B'}}

In [5]:
h.issue('Dave', 'Alice', 'Government Bond 2025', quantity=10, transferable=True, expiry=None, verifiable=True, attributes={'payments': "['10Jun2023', '10Sep2023', '10Dec2023']"})
# ('67461.1681894831.8281841',
#  <Response [200]>,
#  'fe1818dd-0a03-40dd-b367-e71813aad485')

# h.issue('Dave', 'Charlie', 'BE Computers', quantity = None, transferable=False, expiry=None, verifiable=True, attributes={'Year': "2003", 'Name': 'Bob Gupta', 'Marks': '451/500'})

 26-04 12:57:09 | I | Issuing instrument='Government Bond 2025' to 'Alice'
 26-04 12:57:09 | I | Running action 'Issue' on 'Dave/272937EA30B2/CN=Dave, OU=Test Dept, O=R3, L=London, C=GB'
 26-04 12:58:54 | I | 4df1ef96-eeb0-44bf-935e-48cef48753e6


('45337.1682494029.1437619',
 <Response [200]>,
 '4df1ef96-eeb0-44bf-935e-48cef48753e6')

Help on method transfer in module json_on_jupyter:

transfer(inst_id, from_node, to_node, quantity=None) method of json_on_jupyter.JSON_ON_Jupyter instance



,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,7e7bd79d-d5d6-4c9d-954b-9a47e2dc517d,Government Bond 2025,"CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB","CN=Dave, OU=Test Dept, O=R3, L=London, C=GB",3,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023']'}"


128

In [6]:
# h.transfer('fdd47076-7f6a-4ea1-9d1f-c663d9daf97f', 'Alice', 'Bob') # transferable=False
# h.transfer('0697bc47-b1f0-43f5-a37f-841fc353925e', 'Alice', 'Bob') # transferable=True
h.transfer('ab2a0ff2-92c4-4b73-af50-5fed62790214', 'Alice', 'Charlie', 3)

 25-04 11:47:50 | I | Tranfering instrument id 'ab2a0ff2-92c4-4b73-af50-5fed62790214' to 'Charlie'
 25-04 11:47:50 | I | Running action 'Transfer' on 'Alice/45C63C50EA0B/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 25-04 11:49:28 | I | UtxoSignedTransactionImpl(id=SHA-256D:017301B55A8F5B3FC454189BF45BCCC4F3744A23E61E16D80B6A62F83DEFB270, signatures=[DigitalSignatureAndMetadata[signature=[304402203FA9A03DB18B95EB183B7DDDB12C5A6EAF25603DB09149079869A7E1494F939102201C76857B6B47BE6C0D0F874AB66FF1BA327957129E37F6CA693C9AF5E176C711], metadata=DigitalSignatureMetadata[timestamp=2023-04-25T06:17:56.621447Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=null], DigitalSignatureAndMetadata[signature=[304502204310C67DFAB3B1FEB2FE489A460AD24E5D5ADB4CBC6DFA033203FD9CDEEBFE00022100D8924D51BF688CF2D7F945D4735A650A3285FB9A5039545FBD80AC95C838325B], metadata=DigitalSignatureMetadata[timestam

('31099.1682403470.883976',
 <Response [200]>,
 'UtxoSignedTransactionImpl(id=SHA-256D:017301B55A8F5B3FC454189BF45BCCC4F3744A23E61E16D80B6A62F83DEFB270, signatures=[DigitalSignatureAndMetadata[signature=[304402203FA9A03DB18B95EB183B7DDDB12C5A6EAF25603DB09149079869A7E1494F939102201C76857B6B47BE6C0D0F874AB66FF1BA327957129E37F6CA693C9AF5E176C711], metadata=DigitalSignatureMetadata[timestamp=2023-04-25T06:17:56.621447Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=null], DigitalSignatureAndMetadata[signature=[304502204310C67DFAB3B1FEB2FE489A460AD24E5D5ADB4CBC6DFA033203FD9CDEEBFE00022100D8924D51BF688CF2D7F945D4735A650A3285FB9A5039545FBD80AC95C838325B], metadata=DigitalSignatureMetadata[timestamp=2023-04-25T06:18:51.973001Z, signatureSpec=SHA256withECDSA, properties={platformVersion=5000, cpiName=CPI name, cpiVersion=CPI version, cpiSignerSummaryHash=46616B652D76616C7565}], proof=

In [3]:
h.get("Dave", 30020)

 17-04 11:18:18 | I | 200
 17-04 11:18:18 | I | {'holdingIdentityShortHash': '272937EA30B2', 'clientRequestId': '30020', 'flowId': '5af38dc7-0991-4811-86dc-b2c4239a6bf3', 'flowStatus': 'COMPLETED', 'flowResult': '[{"id":"fdd47076-7f6a-4ea1-9d1f-c663d9daf97f","name":"Certificate","owner":"Party(name=CN=Alice, OU=Test Dept, O=R3, L=London, C=GB, owningKey=EC Public Key [ab:e4:40:46:6a:26:a1:3d:60:70:1b:1e:68:a0:81:a9:ca:0f:d2:96]\\n            X: f141c3b2973dce02e8cfeeb3ada735652ce78b55a348b417481c9c6857c6432c\\n            Y: f766dfa43075e5af7c3c89c5aad7af883e9b8e5330a65432240d6d3151daea16\\n)","issuer":"Party(name=CN=Dave, OU=Test Dept, O=R3, L=London, C=GB, owningKey=EC Public Key [28:43:4d:10:b0:6a:bb:88:8a:38:8d:85:1c:81:93:62:88:be:1b:74]\\n            X: 588a1e2d59785a7cf3e37de91f6fbfdcd3af47f5ae62ccf1a84bb4d9365d2a37\\n            Y: 5443ac36886a87b17b61df56c2d18d84e15a0fcf3781ff9bea7ccf7890348cab\\n)","transferable":false,"expiry":null,"attributes":{"college":"Apple Collage","un

(30020, <Response [200]>)

In [5]:
h.log.info("apple")

 17-04 11:03:19 | I | apple


In [4]:
(req, res, result) = h.post('Dave', queryRequest)
h.get("Dave", 30020)

 17-04 11:18:39 | I | 202
 17-04 11:18:39 | I | {'holdingIdentityShortHash': '272937EA30B2', 'clientRequestId': '46048.1681710519.2324579', 'flowId': None, 'flowStatus': 'START_REQUESTED', 'flowResult': None, 'flowError': None, 'timestamp': '2023-04-17T05:48:39.270Z'}
 17-04 11:18:44 | I | 200
 17-04 11:18:44 | I | {'holdingIdentityShortHash': '272937EA30B2', 'clientRequestId': '46048.1681710519.2324579', 'flowId': 'cc8c2422-1dc3-44a1-8421-dafc438f165c', 'flowStatus': 'COMPLETED', 'flowResult': '[{"id":"fdd47076-7f6a-4ea1-9d1f-c663d9daf97f","name":"Certificate","owner":"Party(name=CN=Alice, OU=Test Dept, O=R3, L=London, C=GB, owningKey=EC Public Key [ab:e4:40:46:6a:26:a1:3d:60:70:1b:1e:68:a0:81:a9:ca:0f:d2:96]\\n            X: f141c3b2973dce02e8cfeeb3ada735652ce78b55a348b417481c9c6857c6432c\\n            Y: f766dfa43075e5af7c3c89c5aad7af883e9b8e5330a65432240d6d3151daea16\\n)","issuer":"Party(name=CN=Dave, OU=Test Dept, O=R3, L=London, C=GB, owningKey=EC Public Key [28:43:4d:10:b0:6a:bb

(30020, <Response [200]>)

In [5]:
(req, res) = h.get("Dave", 30020)

 17-04 11:19:21 | I | 200
 17-04 11:19:21 | I | {'holdingIdentityShortHash': '272937EA30B2', 'clientRequestId': '30020', 'flowId': '5af38dc7-0991-4811-86dc-b2c4239a6bf3', 'flowStatus': 'COMPLETED', 'flowResult': '[{"id":"fdd47076-7f6a-4ea1-9d1f-c663d9daf97f","name":"Certificate","owner":"Party(name=CN=Alice, OU=Test Dept, O=R3, L=London, C=GB, owningKey=EC Public Key [ab:e4:40:46:6a:26:a1:3d:60:70:1b:1e:68:a0:81:a9:ca:0f:d2:96]\\n            X: f141c3b2973dce02e8cfeeb3ada735652ce78b55a348b417481c9c6857c6432c\\n            Y: f766dfa43075e5af7c3c89c5aad7af883e9b8e5330a65432240d6d3151daea16\\n)","issuer":"Party(name=CN=Dave, OU=Test Dept, O=R3, L=London, C=GB, owningKey=EC Public Key [28:43:4d:10:b0:6a:bb:88:8a:38:8d:85:1c:81:93:62:88:be:1b:74]\\n            X: 588a1e2d59785a7cf3e37de91f6fbfdcd3af47f5ae62ccf1a84bb4d9365d2a37\\n            Y: 5443ac36886a87b17b61df56c2d18d84e15a0fcf3781ff9bea7ccf7890348cab\\n)","transferable":false,"expiry":null,"attributes":{"college":"Apple Collage","un

In [14]:
pd.DataFrame(json.loads(res.json()['flowResult']))

,id,name,owner,issuer,transferable,expiry,attributes
0,fdd47076-7f6a-4ea1-9d1f-c663d9daf97f,Certificate,"Party(name=CN=Alice, OU=Test Dept, O=R3, L=London, C=GB, owningKey=EC Public Key [ab:e4:40:46:6a:26:a1:3d:60:70:1b:1e:68:a0:81:a9:ca:0f:d2:96]\n X: f141c3b2973dce02e8cfeeb3ada735652ce78b55a348b417481c9c6857c6432c\n Y: f766dfa43075e5af7c3c89c5aad7af883e9b8e5330a65432240d6d3151daea16\n)","Party(name=CN=Dave, OU=Test Dept, O=R3, L=London, C=GB, owningKey=EC Public Key [28:43:4d:10:b0:6a:bb:88:8a:38:8d:85:1c:81:93:62:88:be:1b:74]\n X: 588a1e2d59785a7cf3e37de91f6fbfdcd3af47f5ae62ccf1a84bb4d9365d2a37\n Y: 5443ac36886a87b17b61df56c2d18d84e15a0fcf3781ff9bea7ccf7890348cab\n)",False,None,"{'college': 'Apple Collage', 'university': 'Apple'}"
1,abadf72d-a7b1-43e2-a122-5749dffbee58,Certificate,"Party(name=CN=Alice, OU=Test Dept, O=R3, L=London, C=GB, owningKey=EC Public Key [ab:e4:40:46:6a:26:a1:3d:60:70:1b:1e:68:a0:81:a9:ca:0f:d2:96]\n X: f141c3b2973dce02e8cfeeb3ada735652ce78b55a348b417481c9c6857c6432c\n Y: f766dfa43075e5af7c3c89c5aad7af883e9b8e5330a65432240d6d3151daea16\n)","Party(name=CN=Dave, OU=Test Dept, O=R3, L=London, C=GB, owningKey=EC Public Key [28:43:4d:10:b0:6a:bb:88:8a:38:8d:85:1c:81:93:62:88:be:1b:74]\n X: 588a1e2d59785a7cf3e37de91f6fbfdcd3af47f5ae62ccf1a84bb4d9365d2a37\n Y: 5443ac36886a87b17b61df56c2d18d84e15a0fcf3781ff9bea7ccf7890348cab\n)",False,None,"{'college': 'Apple Collage', 'university': 'Apple'}"


NameError: name 'h' is not defined